In [ ]:
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

     |████████████████████████████████| 435 kB 32.7 MB/s 
     |████████████████████████████████| 482 kB 51.9 MB/s 
     |████████████████████████████████| 462 kB 53.8 MB/s 
     |████████████████████████████████| 41 kB 546 kB/s 


In [ ]:
import pandas as pd
import os
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
 
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1uDs65VW6Jf0O2wdkgtxNl1dmoLm0IUm5/view?usp=sharing'
import pandas as pd
 
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('pages.csv') 
 
df = pd.read_csv('pages.csv')
print(df.head())

   Unnamed: 0                                     Name  \
0           0                           Antiderivative   
1           1                                Hyperbola   
2           2  Area_under_the_curve_(pharmacokinetics)   
3           3         Vector_(mathematics_and_physics)   
4           4                                 Parabola   

                                                Link  Label  
0       https://en.wikipedia.org/wiki/Antiderivative      3  
1            https://en.wikipedia.org/wiki/Hyperbola      3  
2  https://en.wikipedia.org/wiki/Area_under_the_c...      3  
3  https://en.wikipedia.org/wiki/Vector_(mathemat...      1  
4             https://en.wikipedia.org/wiki/Parabola      2  


In [ ]:
from stellargraph import StellarGraph

In [ ]:
df1 = df[['Name','Link']]

In [ ]:
df2 = df1.rename(columns={'Name':'source','Link':'target'})
df2.head()

,source,target
0,Antiderivative,https://en.wikipedia.org/wiki/Antiderivative
1,Hyperbola,https://en.wikipedia.org/wiki/Hyperbola
2,Area_under_the_curve_(pharmacokinetics),https://en.wikipedia.org/wiki/Area_under_the_c...
3,Vector_(mathematics_and_physics),https://en.wikipedia.org/wiki/Vector_(mathemat...
4,Parabola,https://en.wikipedia.org/wiki/Parabola


In [ ]:
graph = StellarGraph(edges=df2)
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 250420, Edges: 126584

 Node types:
  default: [250420]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [126584]
        Weights: all 1 (default)
        Features: none


In [ ]:
square_named = StellarGraph(
    edges=df2, node_type_default="corner", edge_type_default="line"
)
print(square_named.info())

StellarGraph: Undirected multigraph
 Nodes: 250420, Edges: 126584

 Node types:
  corner: [250420]
    Features: none
    Edge types: corner-line->corner

 Edge types:
    corner-line->corner: [126584]
        Weights: all 1 (default)
        Features: none


In [ ]:
from sklearn.preprocessing import LabelEncoder
link = 'https://drive.google.com/file/d/1uDs65VW6Jf0O2wdkgtxNl1dmoLm0IUm5/view?usp=sharing'
import pandas as pd
 
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('pages.csv') 
 
df = pd.read_csv('pages.csv')
print(df.head())

   Unnamed: 0                                     Name  \
0           0                           Antiderivative   
1           1                                Hyperbola   
2           2  Area_under_the_curve_(pharmacokinetics)   
3           3         Vector_(mathematics_and_physics)   
4           4                                 Parabola   

                                                Link  Label  
0       https://en.wikipedia.org/wiki/Antiderivative      3  
1            https://en.wikipedia.org/wiki/Hyperbola      3  
2  https://en.wikipedia.org/wiki/Area_under_the_c...      3  
3  https://en.wikipedia.org/wiki/Vector_(mathemat...      1  
4             https://en.wikipedia.org/wiki/Parabola      2  


In [ ]:
item_encoder = LabelEncoder()
df['Name'] = item_encoder.fit_transform(df.Name)
print(df.head())

   Unnamed: 0    Name                                               Link  \
0           0   16827       https://en.wikipedia.org/wiki/Antiderivative   
1           1   63233            https://en.wikipedia.org/wiki/Hyperbola   
2           2   17742  https://en.wikipedia.org/wiki/Area_under_the_c...   
3           3  118737  https://en.wikipedia.org/wiki/Vector_(mathemat...   
4           4   88457             https://en.wikipedia.org/wiki/Parabola   

   Label  
0      3  
1      3  
2      3  
3      1  
4      2  


In [ ]:
train_subjects, test_subjects = model_selection.train_test_split(
    df['Name'], train_size=140, test_size=None, stratify=df['Name']
)
val_subjects, test_subjects = model_selection.train_test_split(
    test_subjects, train_size=500, test_size=None, stratify=test_subjects
)

ValueError: ignored